In [53]:
# Importing libraries neeeded for the project
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import mysql.connector
import pycountry
import numpy as np


## Connexion à la base pour Wamp

In [54]:
# Connexion à la BD pour Wamp 
host = 'localhost'  
utilisateur = 'root'
mot_de_passe = '' 
base_de_donnees = 'ecotourisme'

# Connexion
connexion = mysql.connector.connect(
    host=host,
    user=utilisateur,
    password=mot_de_passe,
    database=base_de_donnees
)

curseur = connexion.cursor()

if connexion.is_connected():
    print("Connexion à la base de données réussie.")
else:
    print("Échec de la connexion à la base de données.")

Connexion à la base de données réussie.


## Connexion à la base avec Mamp

In [55]:
#!/usr/bin/env /Applications/MAMP/Library/bin/python

# import mysql.connector

# config = {
#   'user': 'root',
#   'password': 'root',
#   'host': '127.0.0.1',
#   'port': 8889,
#   'database': 'ecotourisme',
#   'raise_on_warnings': True
# }

# cnx = mysql.connector.connect(**config)

# curseur = cnx.cursor(dictionary=True)

## Traitement des données

In [56]:


curseur.execute('''
    SELECT 
        p.nom as pays, 
        s.id_pays, 
        s.annee, 
        s.gpi, 
        t.arriveesTotal as arriveesTotal, 
        t.arriveesAvion, 
        e.pibParHab, 
        eco.GES_hab,
        eco.elecRenew,
        d.Value as idh
    FROM 
        surete as s, 
        tourisme as t, 
        economie as e, 
        ecologie as eco, 
        pays as p, 
        developpement_humain as d
    WHERE 
        p.id = s.id_pays 
        AND s.id_pays = t.id_pays 
        AND t.id_pays = e.id_pays 
        AND e.id_pays = eco.id_pays 
        AND d.iso_code = eco.id_pays
        AND s.annee = t.annee 
        AND t.annee = e.annee 
        AND e.annee = eco.annee 
        AND e.annee=2020
        AND d.annee=2021;
''')

dataset = curseur.fetchall()

df = pd.DataFrame(dataset, columns=['pays', 'id_pays', 'annee', 'gpi', 'arriveesTotal', 'arriveesAvion', 'pibParHab', 'GES_hab', 'elecRenew', 'idh'])
df_ecol = df[['pays', 'id_pays', 'annee', 'GES_hab', 'elecRenew']].copy()
df_eco = df[['pays', 'id_pays', 'annee', 'arriveesTotal','pibParHab','idh']].copy()
df_decouverte = df[['pays', 'id_pays', 'annee', 'gpi', 'arriveesTotal', 'arriveesAvion', 'pibParHab', 'GES_hab', 'elecRenew', 'idh']].copy()

# Fonction pour convertir ISO-2 en ISO-3
def iso2_to_iso3(iso2):
    try:
        return pycountry.countries.get(alpha_2=iso2).alpha_3
    except AttributeError:
        return np.nan


## Score global ou score touristique

In [57]:
 
# Normalisation des indicateurs
df['arriveesTotal'] = df['arriveesTotal'] / df['arriveesTotal'].max()
df['gpi'] = df['gpi'].min() / df['gpi']
df['pibParHab'] = df['pibParHab'] / df['pibParHab'].max()
df['GES_hab'] = df['GES_hab'].min() / df['GES_hab']
df['elecRenew'] = df['elecRenew'] / df['elecRenew'].max()
df['idh'] = df['idh'].str.replace(',', '.').astype(float)
df['idh'] = df['idh'] / df['idh'].max()

# Les poids pour chaque variable
poids = {
    'arriveesTotal': 2,
    'gpi': 4,
    'pibParHab': 2,
    'GES_hab': 6, 
    'elecRenew': 3,
    'idh' : 4

}

# Calcul du score touristique 
df['score_touristique'] = df.apply(lambda row: sum(float(row[var]) * poids[var] for var in poids) / sum(poids.values()), axis=1)

df.head()


max_score = df['score_touristique'].max()
min_score = df['score_touristique'].min()

variance = df['score_touristique'].std()
esperance = df['score_touristique'].mean()


# Limites des scores/label
bins = [min_score, 0.30, 0.394, 0.46, 0.53, max_score]

print(max_score, min_score, variance, esperance)

# Les labels 
labels = ['E', 'D', 'C', 'B', 'A']

# Créer une nouvelle colonne score
df['label_touristique'] = pd.cut(df['score_touristique'], bins=bins, labels=labels)

df = df.sort_values(by='score_touristique', ascending=False)

# suppression des valeurs NaN
df.dropna(inplace=True)

# Appliquer la fonction à la colonne 'id_pays'
df['id_pays'] = df['id_pays'].apply(iso2_to_iso3)

# Créer une carte du monde
fig = px.choropleth(df, locations='id_pays', color='label_touristique', hover_name='pays', projection='natural earth')

# Mettre à jour la carte
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue", showlakes=True, lakecolor="Blue")

# Afficher la carte
fig.show()


0.5975751788341688 0.1806876277348624 0.08110374263507238 0.3903514651809921
Nombre total de lignes mises à jour : 141


## Score écologique

In [59]:

# Normalisation des indicateurs par la méthode min-max
df_ecol['GES_hab'] = df_ecol['GES_hab'].min() / df_ecol['GES_hab']
df_ecol['elecRenew'] = df_ecol['elecRenew'] / df_ecol['elecRenew'].max()

# Définir les poids pour chaque variable
poids_ecol = {
    'GES_hab': 3, 
    'elecRenew': 2
}

# Calculer le score écologique 
df_ecol['score_ecologique'] = df_ecol.apply(lambda row: sum(float(row[var]) * poids_ecol[var] for var in poids_ecol) / sum(poids_ecol.values()), axis=1)


df_ecol.head()

max_score_ecol = df_ecol['score_ecologique'].max()
min_score_ecol = df_ecol['score_ecologique'].min()

variance_ecol = df_ecol['score_ecologique'].std()
esperance_ecol = df_ecol['score_ecologique'].mean()

# Les limites des scores/labels
bins = [min_score_ecol, 0.17, 0.34, 0.51, 0.68, max_score_ecol]

print(max_score_ecol, min_score_ecol, variance_ecol, esperance_ecol)

# Les labels
labels = ['E', 'D', 'C', 'B', 'A']

# Créer une nouvelle colonne score
df_ecol['label_ecol'] = pd.cut(df_ecol['score_ecologique'], bins=bins, labels=labels)

df_ecol = df_ecol.sort_values(by='score_ecologique', ascending=False)

df_ecol.head()

df_ecol.dropna(inplace=True)

# Appliquer la fonction à la colonne 'id_pays'
df_ecol['id_pays'] = df_ecol['id_pays'].apply(iso2_to_iso3)

df_ecol.head()

# Créer une carte du monde
fig = px.choropleth(df_ecol, locations='id_pays', color='label_ecol', hover_name='pays', projection='natural earth')

# Mettre à jour la carte
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue", showlakes=True, lakecolor="Blue")

# Afficher la carte
fig.show()


0.8742857199999999 0.0018257330541103425 0.17151731378557097 0.2573908042145492


## Score économique 

In [60]:
# Normalisation des indicateurs par la méthode min-max
df_eco['arriveesTotal'] = df_eco['arriveesTotal'] / df_eco['arriveesTotal'].max()
df_eco['pibParHab'] = df_eco['pibParHab'] / df_eco['pibParHab'].max()
df_eco['idh'] = df_eco['idh'].str.replace(',', '.').astype(float)
df_eco['idh'] = df_eco['idh'] / df_eco['idh'].max()


# Les poids
poids_eco = {
    'arriveesTotal': 2,
    'pibParHab': 2,
    'idh' : 2
}

# Calcul du score économique 
df_eco['score_economique'] = df_eco.apply(lambda row: sum(float(row[var]) * poids_eco[var] for var in poids_eco) / sum(poids_eco.values()), axis=1)
df_eco.head()

max_score_eco = df_eco['score_economique'].max()
min_score_eco = df_eco['score_economique'].min()

variance_eco = df_eco['score_economique'].std()
esperance_eco = df_eco['score_economique'].mean()

# Les limites des scores/labels
bins = [min_score_eco, 0.28, 0.42, 0.56, 0.7, max_score_eco]

print(max_score_eco, min_score_eco, variance_eco, esperance_eco)


# Labels
labels = ['E', 'D', 'C', 'B', 'A']

# création d'une nouvelle colonne pour les labels
df_eco['label_eco'] = pd.cut(df_eco['score_economique'], bins=bins, labels=labels)

df_eco = df_eco.sort_values(by='score_economique', ascending=False)

df_eco.dropna(inplace=True)


# application de la fonction faite précédement 
df_eco['id_pays'] = df_eco['id_pays'].apply(iso2_to_iso3)

df_eco.head()

# Visualisation par une carte
fig = px.choropleth(df_eco, locations='id_pays', color='label_eco', hover_name='pays', projection='natural earth')
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue", showlakes=True, lakecolor="Blue")
fig.show()

0.7951837060465085 0.14186097106176157 0.14087026023151913 0.3572549677903555


## Score découverte 

In [61]:

# Normalisation des indicateurs par la méthode min-max
df_decouverte['arriveesTotal'] = df_decouverte['arriveesTotal'] / df_decouverte['arriveesTotal'].max()
df_decouverte['GES_hab'] = df_decouverte['GES_hab'].min() / df_decouverte['GES_hab']
df_decouverte['idh'] = df_decouverte['idh'].str.replace(',', '.').astype(float)
df_decouverte['idh'] = df_decouverte['idh'] / df_decouverte['idh'].max()


# Les poids pour le tourisme de découverte
poids_decouverte = {
    'arriveesTotal': 2,
    'idh' : 2,
    'GES_hab': 1
}

# Calcul du score pour le tourisme culturel et historique / tourisme d'exploration et de découverte
df_decouverte['score_decouverte'] = df_decouverte.apply(lambda row: sum(float(row[var]) * poids_decouverte[var] for var in poids_decouverte) / sum(poids_decouverte.values()), axis=1)

max_decouverte = df_decouverte['score_decouverte'].max()
min_decouverte = df_decouverte['score_decouverte'].min()
esperance_decouverte = df_decouverte['score_decouverte'].mean()
variance_decouverte = df_decouverte['score_decouverte'].std()


print(max_decouverte, min_decouverte, esperance_decouverte, variance_decouverte )

bins = [min_decouverte, 0.30, 0.40, 0.50, 0.62, max_decouverte]

# Les labels
labels = ['E', 'D', 'C', 'B', 'A']


# Création d'une nouvelle colonne pour les labels
df_decouverte['label_decouverte'] = pd.cut(df_decouverte['score_decouverte'], bins=bins, labels=labels)

df_decouverte = df_decouverte.sort_values(by='score_decouverte', ascending=False)

df_decouverte.dropna(inplace=True)

df_decouverte['id_pays'] = df_decouverte['id_pays'].apply(iso2_to_iso3)

df_decouverte.head()

# Visualisation par une carte
fig = px.choropleth(df_decouverte, locations='id_pays', color='label_decouverte', hover_name='pays', projection='natural earth')
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue", showlakes=True, lakecolor="Blue")
fig.show()

0.7980992787690196 0.1736595576321267 0.36796918619072555 0.08938731100444197
